In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

%load_ext autoreload
%autoreload 2

/home/ylu/anaconda3/envs/SPACEL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import SPACEL
from SPACEL import Scube, Splane

In [3]:
data_folder = "./results/split_slices/"
results_folder = "./results/SPACEL/"
cache_folder = os.path.join(results_folder, 'Scube_outputs')
figures_folder = "./results/figures/SPACEL"
Path(results_folder).mkdir(parents=True, exist_ok=True)
Path(cache_folder).mkdir(parents=True, exist_ok=True)

In [4]:
## load the data
from tqdm import tqdm
slices = []
for i in tqdm(range(59)):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f"slice_{i}.h5ad")))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 59/59 [03:11<00:00,  3.25s/it]


In [5]:
spatial_key = "spatial"
anno_key = "class"
palette = slices[0].uns[f"{anno_key}_palette"]

In [6]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [7]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [8]:
## Perform the SPACEL with provided annotation alignment
sampling_num = 20000
for i in tqdm(range(17, len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    slice1.obsm['spatial_2D'] = slice1.obsm[spatial_key]
    slice2.obsm['spatial_2D'] = slice2.obsm[spatial_key]
    slice1.obsm['spatial'] = slice1.obsm[spatial_key]
    slice2.obsm['spatial'] = slice2.obsm[spatial_key]
    align_slices = [slice1, slice2]
    time_start = time.time()
    Scube.align(
        align_slices,
        cluster_key=anno_key, 
        n_neighbors = 15, 
        n_threads=10,
        p=2,
        write_loc_path=os.path.join(results_folder, 'Scube_outputs/aligned_coordinates.csv')
    )
    align_slices[0].obsm[key_added] = align_slices[0].obsm['spatial_aligned'].values
    align_slices[1].obsm[key_added] = align_slices[1].obsm['spatial_aligned'].values
    time_end = time.time()
    
    R1, t1=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    R2, t2=solve_RT_by_correspondence(align_slices[0].obsm[spatial_key], align_slices[0].obsm[key_added])
    t = t1 @ R2.T + t2
    R = R2 @ R1
    alignment_results = {'R': R, 't': t, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                                                                                                                                     | 0/41 [00:00<?, ?it/s]

Start alignment...
Alignment slice 1 to 0
Runtime: 142.24168515205383 s


  2%|███▎                                                                                                                                      | 1/41 [02:22<1:35:17, 142.95s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 180.70523023605347 s


  5%|██████▋                                                                                                                                   | 2/41 [05:24<1:47:37, 165.58s/it]

...

 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎      | 39/41 [2:00:19<06:41, 200.50s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 233.08578777313232 s


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋   | 40/41 [2:04:13<03:30, 210.49s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 129.1465606689453 s


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [2:06:23<00:00, 184.96s/it]
